In [2]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.optimizers import SGD
import pandas as pd
from sklearn.model_selection import train_test_split
import math

n_in = 5

In [3]:
df = pd.read_csv('../data/crawler/unified-events-statistics.csv')
df = df.drop(['event1time','event2time','event3time','event4time','event5time','event6time','event7time','event8time','event9time','event10time','event11time','event12time','event13time','event14time','event15time','event16time','event17time','event18time','event19time','event20time','event21time','event22time','event23time','event24time','event25time','event26time','event27time','event28time','event29time','event30time','event31time','event32time','event33time','event34time','event35time','event36time','event37time','event38time','event39time','event40time','event41time','event42time','event43time','event44time','event45time','blueTopGP','blueTopWR','blueTopKDA','blueJungleGP','blueJungleWR','blueJungleKDA','blueMidGP','blueMidWR','blueMidKDA','blueADCGP','blueADCWR','blueADCKDA','blueSupportGP','blueSupportWR','blueSupportKDA','redTopGP','redTopWR','redTopKDA','redJungleGP','redJungleWR','redJungleKDA','redMidGP','redMidWR','redMidKDA','redAdcGP','redAdcWR','redAdcKDA','redSupportGP','redSupportWR','redSupportKDA','result','qtd_eventos','duracao_min','golId','game'], axis=1)
df = df.replace(['BLUE: first_blood',
    'BLUE: dragon',
    'BLUE: herald',
    'BLUE: first_tower_top',
    'BLUE: first_tower_mid',
    'BLUE: first_tower_bot',
    'BLUE: second_tower_top',
    'BLUE: second_tower_mid',
    'BLUE: second_tower_bot',
    'BLUE: third_tower_top',
    'BLUE: third_tower_mid',
    'BLUE: third_tower_bot',
    'BLUE: inhibitor_top',
    'BLUE: inhibitor_mid',
    'BLUE: inhibitor_bot',
    'BLUE: baron',
    'BLUE: elder_dragon',
    'BLUE: nexus_tower',
    'BLUE: nexus',
    'RED: first_blood',
    'RED: dragon',
    'RED: herald',
    'RED: first_tower_top',
    'RED: first_tower_mid',
    'RED: first_tower_bot',
    'RED: second_tower_top',
    'RED: second_tower_mid',
    'RED: second_tower_bot',
    'RED: third_tower_top',
    'RED: third_tower_mid',
    'RED: third_tower_bot',
    'RED: inhibitor_top',
    'RED: inhibitor_mid',
    'RED: inhibitor_bot',
    'RED: baron',
    'RED: elder_dragon',
    'RED: nexus_tower',
    'RED: nexus'], range(38))
df

,event1,event2,event3,event4,event5,event6,event7,event8,event9,event10,...,event36,event37,event38,event39,event40,event41,event42,event43,event44,event45
0,19,2,20,5,20,2,24,4,22,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,19,5,20,4,22,21,23,24,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,1,21,5,1,21,3,20,23,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,19,21,1,2,24,1,4,3,7,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19,21,1,22,1,24,27,2,23,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6009,19,1,2,20,22,24,21,23,1,34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6010,19,20,21,20,21,5,23,20,26,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6011,20,0,21,1,22,1,5,21,3,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6012,20,21,0,20,21,5,20,22,23,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
def splitSequence(games, n_steps_in, n_steps_out=2):
    X, Y = [], []
    
    for sequence in games:
        seq_y = 0 if sequence[-1] == 37 else 1
        for i in range(len(sequence)):
            # find the end of this pattern
            end_ix = i + n_steps_in
            out_end_ix = end_ix + n_steps_out
            # check if we are beyond the sequence
            if out_end_ix > len(sequence):
                break
            # gather input and output parts of the pattern
            seq_x = sequence[i:end_ix]
            X.append(seq_x)
            Y.append(seq_y)
	
    return X, Y

lista_eventos = df.values.tolist()

data = [[int(elemento) for elemento in sublista if not math.isnan(elemento)] for sublista in lista_eventos]
data = np.array(data)
data.size

c:\Users\linco\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


6014

In [5]:
x_train, x_test = train_test_split(data,test_size=0.2,stratify=y)
x_train

array([list([19, 2, 20, 5, 20, 2, 24, 4, 22, 20, 23, 1, 34, 26, 29, 32, 20, 25, 34, 3, 27, 30, 33, 32, 28, 31, 36, 36, 37]),
       list([2, 19, 5, 20, 4, 22, 21, 23, 24, 3, 27, 20, 34, 25, 8, 26, 6, 1, 7, 34, 1, 30, 33, 29, 32, 1, 11, 15, 20, 14, 20, 32, 36, 36, 37]),
       list([0, 1, 21, 5, 1, 21, 3, 20, 23, 22, 1, 8, 4, 15, 1, 7, 6, 24, 27, 16, 34, 9, 12, 10, 13, 17, 17, 18]),
       ...,
       list([20, 2, 19, 20, 3, 21, 23, 5, 26, 24, 20, 4, 15, 20, 22, 34, 29, 32, 36, 36, 37]),
       list([20, 2, 0, 20, 21, 22, 1, 23, 20, 24, 4, 15, 3, 5, 7, 8, 10, 13, 17, 17, 18]),
       list([0, 20, 2, 1, 3, 24, 2, 20, 23, 4, 5, 34, 22, 20, 27, 26, 30, 33, 29, 32, 36, 36, 37])],
      dtype=object)

In [6]:
train, y_train = splitSequence(x_train, n_in)
y_train

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [7]:
test, y_test = splitSequence(x_test, n_in)

In [9]:
# Definindo o modelo
model = Sequential([
    Embedding(input_dim=38, output_dim=16, input_length=max_length-1),
    SimpleRNN(units=32),
    Dense(units=1, activation='sigmoid')  # A saída é agora uma única unidade com sigmoid
])

# Compilando o modelo
sgd = SGD(learning_rate=0.001, momentum=0.3)
model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])

# Treinando o modelo
model.fit(train, y_train, epochs=10, batch_size=32)

Epoch 1/10
2871/2871 [==============================] - 13s 4ms/step - loss: 0.6640 - accuracy: 0.6561
Epoch 2/10
2871/2871 [==============================] - 10s 4ms/step - loss: 0.5059 - accuracy: 0.7732
Epoch 3/10
2871/2871 [==============================] - 11s 4ms/step - loss: 0.4404 - accuracy: 0.7975
Epoch 4/10
2871/2871 [==============================] - 11s 4ms/step - loss: 0.4244 - accuracy: 0.8055
Epoch 5/10
2871/2871 [==============================] - 10s 3ms/step - loss: 0.4168 - accuracy: 0.8093
Epoch 6/10
2871/2871 [==============================] - 9s 3ms/step - loss: 0.4120 - accuracy: 0.8117
Epoch 7/10
2871/2871 [==============================] - 9s 3ms/step - loss: 0.4089 - accuracy: 0.8136
Epoch 8/10
2871/2871 [==============================] - 9s 3ms/step - loss: 0.4068 - accuracy: 0.8143
Epoch 9/10
2871/2871 [==============================] - 9s 3ms/step - loss: 0.4053 - accuracy: 0.8157
Epoch 10/10
2871/2871 [==============================] - 9s 3ms/step - loss: 

In [10]:
y_test

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [11]:
y_pred = model.predict(test)
y_true = y_test

505/505 [==============================] - 1s 2ms/step


In [18]:
y_pred[5]

array([0.34606588], dtype=float32)

In [23]:
# Passo 1: Obter Previsões e Rótulos Verdadeiros
y_pred[y_pred <= 0.5] = int(0)
y_pred[y_pred > 0.5] = int(1)
y_true = y_test
y_pred[5]

array([0.], dtype=float32)

In [41]:
len(y_true)

16160

In [40]:
index_erros = []

for idx, previsao_modelo in enumerate(y_pred_classes):
    if int(previsao_modelo) != y_true[idx]:
        index_erros.append(idx)

len(index_erros)

2837

In [45]:
exemplos_com_erros = [test[i] for i in index_erros]
rótulos_com_erros = [y_true[i] for i in index_erros]
previsões_com_erros = [y_pred_classes[i] for i in index_erros]

# Passo 4: Analisar os Erros
for i, indice_erro in enumerate(index_erros):
    print(f"Exemplo {i+1}:")
    print(f"   Previsão: {previsões_com_erros[i]}, Rótulo Verdadeiro: {rótulos_com_erros[i]}")
    print(f"   Dados: {exemplos_com_erros[i]}")

# Passo 5: Calcular Métricas de Desempenho
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes)
recall = recall_score(y_true, y_pred_classes)
f1 = f1_score(y_true, y_pred_classes)

print(f"Acurácia: {accuracy}")
print(f"Precisão: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")


Exemplo 1:
   Previsão: [0], Rótulo Verdadeiro: 1
   Dados: [2, 4, 34, 1, 23]
Exemplo 2:
   Previsão: [0], Rótulo Verdadeiro: 1
   Dados: [4, 34, 1, 23, 26]
Exemplo 3:
   Previsão: [0], Rótulo Verdadeiro: 1
   Dados: [34, 1, 23, 26, 7]
Exemplo 4:
   Previsão: [0], Rótulo Verdadeiro: 1
   Dados: [24, 1, 23, 21, 5]
Exemplo 5:
   Previsão: [0], Rótulo Verdadeiro: 1
   Dados: [1, 23, 21, 5, 22]
Exemplo 6:
   Previsão: [0], Rótulo Verdadeiro: 1
   Dados: [23, 21, 5, 22, 34]
Exemplo 7:
   Previsão: [0], Rótulo Verdadeiro: 1
   Dados: [21, 5, 22, 34, 27]
Exemplo 8:
   Previsão: [0], Rótulo Verdadeiro: 1
   Dados: [5, 22, 34, 27, 20]
Exemplo 9:
   Previsão: [0], Rótulo Verdadeiro: 1
   Dados: [22, 34, 27, 20, 25]
Exemplo 10:
   Previsão: [0], Rótulo Verdadeiro: 1
   Dados: [34, 27, 20, 25, 15]
Exemplo 11:
   Previsão: [1], Rótulo Verdadeiro: 0
   Dados: [19, 1, 2, 20, 5]
Exemplo 12:
   Previsão: [1], Rótulo Verdadeiro: 0
   Dados: [1, 2, 20, 5, 2]
Exemplo 13:
   Previsão: [1], Rótulo Verdadeir